In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, roc_auc_score
import scipy.stats as stats
from pathlib import Path

# Create output directory if it doesn't exist
Path("../data/postfilter_data").mkdir(parents=True, exist_ok=True)

# Load data ---------------------------------------------------------------
# Load the expression level data. TPM and readCounts, the first column contains gene IDs
tpm = pd.read_csv("../data/prefilter_data/pnas_tpm_96_nodup.txt", sep="\t", header=None)
readcounts = pd.read_csv("../data/prefilter_data/pnas_readcounts_96_nodup.txt", sep="\t", header=None)


# Load sample meta data file
# the column "sample_id" matches the column names of tpm and readcounts datasets
# the column "recurStatus" shows the recurrence status of the patient, R = recurrence, N = non-recurrence
patient_info = pd.read_csv("../data/prefilter_data/pnas_patient_info.csv")


# Extract sample IDs and recurrence status from patient_info
sample_ids = patient_info['sample_id'].values
recur_status = patient_info['recurStatus'].values

# Create sample_recurStatus dataframe with sample IDs and recurrence status
sample_recurStatus = pd.DataFrame({
    'sample_id': sample_ids,
    'recurStatus': recur_status
})

print(sample_recurStatus)

# Add header row to readcounts and tpm
readcounts.columns = ['gene_id'] + list(sample_ids)
tpm.columns = ['gene_id'] + list(sample_ids)



   sample_id recurStatus
0    S01_B14           R
1    S02_B14           R
2    S03_B14           R
3    S04_B14           R
4    S05_B14           R
..       ...         ...
91   S92_B14           N
92   S93_B14           N
93   S94_B14           N
94   S95_B14           N
95   S96_B14           N

[96 rows x 2 columns]


In [4]:
# Load the pre-selected marker gene list
with open("../data/prefilter_data/preselectedList.txt", "r") as f:
    preselectedList = [line.strip() for line in f.readlines()]

# Differential expression (DE) analysis of the biomarker genes ---------------
# Use scipy.stats to calculate the p-values for each biomarker gene
# Filter readcounts for preselected genes
filtered_readcounts = readcounts[readcounts['gene_id'].isin(preselectedList)].copy()
filtered_readcounts = filtered_readcounts.set_index('gene_id')

# Split samples by recurrence status
recur_samples = sample_recurStatus[sample_recurStatus['recurStatus'] == 'R']['sample_id'].tolist()
nonrecur_samples = sample_recurStatus[sample_recurStatus['recurStatus'] == 'N']['sample_id'].tolist()

# Calculate p-values using t-test
pvalues = []
gene_ids = []

for gene in filtered_readcounts.index:
    recur_vals = filtered_readcounts.loc[gene, recur_samples].values
    nonrecur_vals = filtered_readcounts.loc[gene, nonrecur_samples].values
    _, pval = stats.ttest_ind(recur_vals, nonrecur_vals)
    pvalues.append(pval)
    gene_ids.append(gene)

# Create results dataframe and rank genes by p-value
res = pd.DataFrame({'id': gene_ids, 'pval': pvalues})
rank_marker_gene = res.sort_values('pval')['id'].tolist()

# Ensure output directory exists
Path("../data/postfilter_data/recurrence").mkdir(parents=True, exist_ok=True)

# Filter readcounts and tpm for ranked marker genes and save to output directory
filtered_readcounts_ranked = readcounts[readcounts['gene_id'].isin(rank_marker_gene)].copy()
filtered_readcounts_ranked = filtered_readcounts_ranked.set_index('gene_id').loc[rank_marker_gene].reset_index()
filtered_readcounts_ranked.to_csv("../data/postfilter_data/recurrence/readcounts_filtered_ranked.txt", sep="\t", index=False)

filtered_tpm_ranked = tpm[tpm['gene_id'].isin(rank_marker_gene)].copy()
filtered_tpm_ranked = filtered_tpm_ranked.set_index('gene_id').loc[rank_marker_gene].reset_index()
filtered_tpm_ranked.to_csv("../data/postfilter_data/recurrence/tpm_filtered_ranked.txt", sep="\t", index=False)



In [3]:
# Save sample_recurStatus to the postfilter_data directory
sample_recurStatus.to_csv("../data/postfilter_data/recurrence/sample_recurStatus.txt", sep="\t", index=False)

In [ ]:
# Read in the normal sample read count and TPM data
normal_readcounts = pd.read_csv("../data/prefilter_data/pnas_normal_readcounts.txt", sep="\t")
normal_tpm = pd.read_csv("../data/prefilter_data/pnas_normal_tpm.txt", sep="\t")


In [22]:
# Load the pre-selected marker gene list
# Differential expression (DE) analysis of the biomarker genes: normal vs 96 samples

# Filter readcounts for preselected genes
filtered_readcounts_96 = readcounts[readcounts['gene_id'].isin(preselectedList)].copy()
filtered_readcounts_96 = filtered_readcounts_96.set_index('gene_id')
filtered_readcounts_normal = normal_readcounts[normal_readcounts['gene_id'].isin(preselectedList)].copy()
filtered_readcounts_normal = filtered_readcounts_normal.set_index('gene_id')

# Get sample columns for 96 and normal
sample_96_cols = list(sample_ids)
sample_normal_cols = [col for col in normal_readcounts.columns if col != 'gene_id']

# Calculate p-values using t-test for each gene
pvalues_normal_vs_96 = []
gene_ids_normal_vs_96 = []

for gene in filtered_readcounts_96.index:
    vals_96 = filtered_readcounts_96.loc[gene, sample_96_cols].values
    vals_normal = filtered_readcounts_normal.loc[gene, sample_normal_cols].values
    _, pval = stats.ttest_ind(vals_normal, vals_96, nan_policy='omit')
    pvalues_normal_vs_96.append(pval)
    gene_ids_normal_vs_96.append(gene)

# Create results dataframe and rank genes by p-value
res_normal_vs_96 = pd.DataFrame({'id': gene_ids_normal_vs_96, 'pval': pvalues_normal_vs_96})
rank_marker_gene_normal_vs_96 = res_normal_vs_96.sort_values('pval')['id'].tolist()

# Ensure output directory exists for cancer
Path("../data/postfilter_data/cancer").mkdir(parents=True, exist_ok=True)

# Filter and save ranked readcounts and tpm for normal vs 96
filtered_readcounts_ranked_normal_vs_96 = readcounts[readcounts['gene_id'].isin(rank_marker_gene_normal_vs_96)].copy()
filtered_readcounts_ranked_normal_vs_96 = filtered_readcounts_ranked_normal_vs_96.set_index('gene_id').loc[rank_marker_gene_normal_vs_96]

# Add normal columns for the same genes, matching order
normal_counts_matched = normal_readcounts[normal_readcounts['gene_id'].isin(rank_marker_gene_normal_vs_96)].copy()
normal_counts_matched = normal_counts_matched.set_index('gene_id').loc[rank_marker_gene_normal_vs_96]

# Concatenate 96 samples and normal samples
readcounts_concat = pd.concat([filtered_readcounts_ranked_normal_vs_96, normal_counts_matched[sample_normal_cols]], axis=1).reset_index()
readcounts_concat.to_csv("../data/postfilter_data/cancer/readcounts_filtered_ranked_normal_vs_96.txt", sep="\t", index=False)

filtered_tpm_ranked_normal_vs_96 = tpm[tpm['gene_id'].isin(rank_marker_gene_normal_vs_96)].copy()
filtered_tpm_ranked_normal_vs_96 = filtered_tpm_ranked_normal_vs_96.set_index('gene_id').loc[rank_marker_gene_normal_vs_96]

# Add normal TPM columns for the same genes, matching order
normal_tpm_matched = normal_tpm[normal_tpm['gene_id'].isin(rank_marker_gene_normal_vs_96)].copy()
normal_tpm_matched = normal_tpm_matched.set_index('gene_id').loc[rank_marker_gene_normal_vs_96]

# Concatenate 96 samples and normal samples
tpm_concat = pd.concat([filtered_tpm_ranked_normal_vs_96, normal_tpm_matched[sample_normal_cols]], axis=1).reset_index()
tpm_concat.to_csv("../data/postfilter_data/cancer/tpm_filtered_ranked_normal_vs_96.txt", sep="\t", index=False)

In [23]:
# Create a DataFrame indicating whether each sample is cancerous (in 96) or normal
sample_status = []

# Add 96 cancer samples
for sid in sample_recurStatus['sample_id']:
    sample_status.append({'sample_id': sid, 'status': 'cancer'})

# Add normal samples
for sid in sample_normal_cols:
    sample_status.append({'sample_id': sid, 'status': 'normal'})

sample_status_df = pd.DataFrame(sample_status)
sample_status_df.to_csv("../data/postfilter_data/cancer/sample_status.txt", sep="\t", index=False)